### 1 - Import test data

In [15]:
import datetime as dt
import yfinance as yf
import pandas as pd
import ta

#this is a rest API - it only gets data every 15mins and acts off what is finds

# IMPORTANT changed DNS settings can effect this (will stop it working)

x = dt.datetime.now()
nowdate = f'{x.strftime("%Y")}-{x.strftime("%m")}-{x.strftime("%d")}'

date_60_days_ago = x - dt.timedelta(days=59)
formatted_date = f"{date_60_days_ago.strftime('%Y')}-{date_60_days_ago.strftime('%m')}-{date_60_days_ago.strftime('%d')}"

fifteen_min_dataF = yf.download("EURUSD=X", start=formatted_date, end=nowdate, interval='15m')
fifteen_min_dataF.iloc[:,:]
#fifteen_min_dataF.Open.iloc

one_day_dataF = yf.download('EURUSD=X', period='1y', interval='1d')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


### 2 - Define your signal functions

In [22]:
def og_signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]
    
    # Bearish Pattern
    if (open>close and 
    previous_open<previous_close and 
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    elif (open<close and 
        previous_open>previous_close and 
        close>previous_open and
        open<=previous_close):
        return 2
    
    # No clear pattern
    else:
        return 0


In [31]:
def sma_signal_generator(df):
    #very simple process
    sma_50 = ta.trend.SMAIndicator(df.Close, 50).sma_indicator()
    sma_200 = ta.trend.SMAIndicator(close=df.Close, window=200).sma_indicator()
    #bearish
    if sma_50 > sma_200:
        return 2
    #bullish
    elif sma_50 < sma_200:
        return 1
    #none
    else:
        return 0



#### add signal column

In [23]:
signal = []
signal.append(0)
for i in range(1,len(fifteen_min_dataF)):
    df = fifteen_min_dataF[i-1:i+1]
    signal.append(og_signal_generator(df))
#og_signal_generator(data)
fifteen_min_dataF["signal"] = signal


In [25]:
fifteen_min_dataF.signal.value_counts()
fifteen_min_dataF.iloc[:, :]

,Open,High,Low,Close,Adj Close,Volume,signal
Datetime,,,,,,,
2024-06-03 00:00:00,1.085423,1.085541,1.085423,1.085423,1.085423,0,0
2024-06-03 00:15:00,1.085423,1.085423,1.085305,1.085423,1.085423,0,0
2024-06-03 00:30:00,1.085423,1.085658,1.085423,1.085423,1.085423,0,0
2024-06-03 00:45:00,1.085423,1.085541,1.085423,1.085541,1.085541,0,0
2024-06-03 01:00:00,1.085541,1.085658,1.085423,1.085658,1.085658,0,0
...,...,...,...,...,...,...,...
2024-07-31 22:45:00,1.083189,1.083189,1.082954,1.083072,1.083072,0,0
2024-07-31 23:00:00,1.082954,1.083424,1.082954,1.083189,1.083189,0,0
2024-07-31 23:15:00,1.083189,1.083189,1.082954,1.083072,1.083072,0,0


In [26]:
#checking the return on the 1 day data download

signal = []
signal.append(0)
for i in range(1,len(one_day_dataF)):
    df = one_day_dataF[i-1:i+1]
    signal.append(og_signal_generator(df))
#og_signal_generator(data)

one_day_dataF["signal"] = signal
one_day_dataF.signal.value_counts()
one_day_dataF.iloc[:, :]

,Open,High,Low,Close,Adj Close,Volume,signal
Date,,,,,,,
2023-08-01,1.099759,1.100231,1.095446,1.099759,1.099759,0,0
2023-08-02,1.100788,1.100703,1.091977,1.100788,1.100788,0,0
2023-08-03,1.094415,1.095398,1.091358,1.094415,1.094415,0,0
2023-08-04,1.095158,1.104094,1.093625,1.095158,1.095158,0,0
2023-08-07,1.100267,1.101200,1.096611,1.100267,1.100267,0,0
...,...,...,...,...,...,...,...
2024-07-26,1.085128,1.086850,1.084269,1.085128,1.085128,0,0
2024-07-29,1.085376,1.087193,1.080310,1.085376,1.085376,0,0
2024-07-30,1.081947,1.083776,1.079995,1.081947,1.081947,0,0


In [5]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [6]:
def str_to_bool(s):
    if s.lower() in ['true', '1', 't', 'y', 'yes']:
        return True
    elif s.lower() in ['false', '0', 'f', 'n', 'no']:
        return False
    else:
        raise ValueError(f"Cannot convert {s} to boolean, please input a formattable string")

In [7]:
is_live = False
has_prompted = False

def auth_deets(is_live, type_:str,):
    global has_prompted
    if not has_prompted:
        is_live = str_to_bool(s=input("Would you like to run on your LIVE account? if n then will be DEMO (y/n)"))
        has_prompted = True
    if is_live:
        if type_ == "token":
            access_token='84aa14ef906b9b989c64bff10d032b30-7fefd28cd58969179eaf00aad7c2baa5'# you need token here generated from OANDA account for LIVE
            return access_token
        elif type_ == "id":
            accID = '001-004-12283531-001' #my account ID here from oanda for LIVE
            return accID
    else:
        if type_ == "token":
            access_token='eac0b37f2067f37b1bb9884dfb473e6b-e204948541e0c8a2ba8a967492225ca7' # DEMO TOKEN
            return access_token
        elif type_ == "id":
            accID = '101-004-29576199-001' #my acc ID for the DEMO
            return accID

In [8]:
#from config import access_token, accountID
x=0
def M15_get_candles(n, access_token=auth_deets(is_live, "token"), accountID=auth_deets(is_live, "id")):
    client = CandleClient(access_token, real=is_live)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15) #15 is the time frame (15minutes)
    candles = collector.grab(n)
    return candles#<- contain info like open and close price

#in the "oanda_candles" in the env packages it shows the bot will either use the api url for real or practice trading depending on the real statement in the client line

candles = M15_get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)
# TRUE outputs tell us the last candles have an open price above 1

True
True
True


#### 3 - Connect to the market and execute trades

In [11]:
def M15_trading_job():
    candles = M15_get_candles(3) #last candle is still open (its a current price so the close price is fluctuating)
    dfstream = pd.fifteen_min_dataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = og_signal_generator(dfstream.iloc[:-1,:])
    
    # EXECUTING ORDERS

    accID = auth_deets(is_live, "id")
    access_token = auth_deets(is_live, "token")

    client = API(access_token)

    #-----------------------------------------------------------------
    # all this defines stop loss and stop profit     
    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])
    
    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio
    
    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)

    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID = accID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID = accID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 4 - Executing orders automatically with a scheduler

In [12]:
M15_trading_job()

#scheduler = BlockingScheduler()
#scheduler.add_job(M15_trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
#scheduler.start()

      Open    Close     High      Low
0  1.07879  1.07837  1.07902  1.07836
1  1.07836  1.07826  1.07837  1.07778
1.0794400000000002    1.07767    1.0770799999999998    1.07885
